In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 

def get_category_popular_keyword_naver(category_num):
    driver_location = '.\chromedriver'
    
    return_data = list()
    driver = webdriver.Chrome(executable_path=driver_location)
    
    driver.get('https://datalab.naver.com/')
    
    select_list = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".select_btn")))
    select_list.click()
    time.sleep(1)
    
    select_list = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.select.depth._dropdown > ul > li:nth-child(" + str(category_num) + ") > a")))
    select_list.click()
    time.sleep(2)
    
    datas1 = driver.find_elements(By.CSS_SELECTOR, "div.keyword_rank div.rank_scroll li.list span.title")
    for item in datas1:
        return_data.append(item.text)
    
    driver.quit()
        
    return return_data[-10:]

In [2]:
import requests
from bs4 import BeautifulSoup

def get_related_keyword_google(keyword):
    return_data = list()
    google_related_keyword_api = 'http://suggestqueries.google.com/complete/search?output=toolbar&q=' + keyword
    res = requests.get(google_related_keyword_api)
    
    soup = BeautifulSoup(res.content, 'xml')
    datas1 = soup.select('suggestion')
    
    for item in datas1:
        return_data.append(item['data'])
        
    return return_data

In [3]:
import settings # settings.py 필요
import requests

client_id = settings.NAVER_CLIENT_ID # API access key
client_secret = settings.NAVER_CLIENT_SECRET # API secret key

def naver_shopping_api_template(naver_open_api, body):
    header_params = {"X-Naver-Client-Id": client_id, "X-Naver-Client-Secret": client_secret, "Content-Type":"application/json"}
    res = requests.post(naver_open_api, headers=header_params, data=body)

    if res.status_code == 200:
        return res.json()
    else:
        print('Error Code: ', res.status_code)
        return None

In [4]:
def generate_data(data_json):
    keword_datas = list()
    
    for item in data_json['result']:
        keword_data = list()
        keword_data_ratio = 0
        
        for item2 in item['data']:
            keword_data_ratio += item2['ratio']
            
        keword_data.append(keword_data_ratio)
        keword_data.append(item2['title'])
        
        keword_datas.append(keword_data)
        
    keword_datas.sort(reverse=True)
    print(keword_datas)
    return keword_datas[1][1]